In [1]:
import random

def hex_code_color():
    a = hex(random.randrange(0,256))
    b = hex(random.randrange(0,256))
    c = hex(random.randrange(0,256))
    a = a[2:]
    b = b[2:]
    c = c[2:]
    if len(a)<2:
        a = "0" + a
    if len(b)<2:
        b = "0" + b
    if len(c)<2:
        c = "0" + c
    z = a + b + c
    return "#" + z.upper()

In [9]:
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import DatetimeTickFormatter
import numpy as np
output_notebook()

# plotter
def SMA_plotter(code, df, x_axis_label):
    # create a new plot
    p = figure(title=code + " Simple Moving Averages", x_axis_label=x_axis_label, y_axis_label='Closing Price', width=800, x_axis_type='datetime')

    x_index = np.where(df.columns.values==x_axis_label)
    columns = np.delete(df.columns.values, x_index)
    print columns
    for item in columns:
        color = hex_code_color()
        if item == 'Close':
            p.line(df[x_axis_label], df[item], legend=item, line_color=color, line_width=3)
        else:
            p.line(df[x_axis_label], df[item], legend=item, line_color=color)
    
    p.xaxis.formatter=DatetimeTickFormatter(formats=dict(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    ))

    show(p, notebook_handle=True)

Loading BokehJS ...

In [55]:
# %load Moving_Average.py
"""
Created on Tue Feb 07 23:30:28 2017

@author: rkprajap
"""
# import os, datetime, logging
# import pandas as pd
# import glob
# import numpy as np

# cur_dir = os.getcwd()
# today = datetime.date.today()
# FORMAT = '%(asctime)s - [%(levelname)s]: %(message)s'
# log_file = 'log\\' + str(today) + '.log'
# logging.basicConfig(format=FORMAT, filename=log_file, filemode='w', level=logging.DEBUG, datefmt='%d-%B-%Y %I:%M:%S %p')

# data_files = glob.glob("data\\*.xlsx")
# sma_list = [5, 10, 20, 50]
# scored_df = pd.DataFrame(columns=['Code']).set_index(['Code']) #this df contains final scoring


# calculate SMA scores from a given SMA list
def compute_SMA_scores(smalist, code, df):
    for sma in sma_list:
        df[str(sma)] = df['Close'].rolling(window=sma,center=False).mean()

    final_sma_values = df[[str(item) for item in smalist]].tail(1).values[0]
    sorted_sma = np.sort(sma_list)
    
    score = 0
    for index, sma in enumerate(sorted_sma):
        if df['Close'].values[0] < final_sma_values[index]:
            score = score + (index + 1)*sma
    temp_df = pd.DataFrame([[code, score]], columns=['Code', 'SMA_Score']).set_index(['Code'])
    return temp_df

#         df = stock_history[sma_columns].tail(np.max(sma_list))
#         print code, score, df['Close'].values[0] < df['SMA5'].values[0], df['Close'].values[0] < df['SMA10'].values[0], df['Close'].values[0] < df['SMA20'].values[0], df['Close'].values[0] < df['SMA50'].values[0]
#         SMA_plotter(code, df, 'Date')

In [85]:
def compute_yearly_high_low_scores(df):
    df['52WH_Score'] = 0.0
    df['52WL_Score'] = 0.0
    df = df.tail(365)

    highest_value = df['Close'].max()
    lowest_value = df['Close'].min()
    current_value = df['Close'].tail(1).values[0]
        
    gap_2_highest = round(((highest_value - current_value)/current_value)*100,2)
    gap_2_lowest = round(((current_value - lowest_value)/current_value)*100, 2)
        
    return gap_2_highest, gap_2_lowest

In [82]:
def compute_stock_scores(code, df):
#     SMA Scores
    sma_list = [5, 10, 20, 50]
    result = compute_SMA_scores(sma_list, code, df)
    
# 365 days high and low difference score
    yearly_hl_score = compute_yearly_high_low_scores(df)
    result.set_value(code, '365H_Score', yearly_hl_score[0])
    result.set_value(code, '365L_Score', yearly_hl_score[1])
    return result
    

In [86]:
import os, datetime, logging
import pandas as pd
import glob
import numpy as np

cur_dir = os.getcwd()
today = datetime.date.today()
FORMAT = '%(asctime)s - [%(levelname)s]: %(message)s'
log_file = 'log\\' + str(today) + '.log'
logging.basicConfig(format=FORMAT, filename=log_file, filemode='w', level=logging.DEBUG, datefmt='%d-%B-%Y %I:%M:%S %p')

data_files = glob.glob("data\\*.xlsx")
sma_list = [5, 10, 20, 50]
scored_df = pd.DataFrame(columns=['Code']).set_index(['Code']) #this df contains final scoring

# Start computing stats for each stock
for index, f in enumerate(data_files):
    try:
        code = f.replace('data\\', '').replace('.xlsx', '')
        stock_history = pd.read_excel(f, sheetname='Daily Data', header=0)
        stock_df = compute_stock_scores(code, stock_history)
        scored_df = scored_df.append(stock_df)
    except RuntimeError as err:
        logging.error('Could not calculate score for ' + code + ' - ' + str(err))
    
# normalize the scores
for column in scored_df:
    scored_df[column] = (scored_df[column]/np.max(scored_df[column])) * 10

# for buy decision, high SMA score desired, high 52WH score desired, low 52WL score desired
buy_order = scored_df.sort_values(by=['SMA_Score','365H_Score', '365L_Score'], ascending=[False, False, True])
print buy_order

            SMA_Score  365H_Score  365L_Score
Code                                         
IOC              10.0    0.729063    3.262215
TORNTPHARM       10.0    0.370219    0.237060
GLENMARK         10.0    0.310479    3.492691
ASIANPAINT       10.0    0.241211    2.631371
ASHOKLEY         10.0    0.214159    2.487818
DABUR            10.0    0.189157    1.606743
SIEMENS          10.0    0.173991    2.443040
HEROMOTOCO       10.0    0.151551    3.861451
PEL              10.0    0.143763    7.246148
SHREECEM         10.0    0.136078    5.269327
EMAMILTD         10.0    0.127471    2.834189
LICHSGFIN        10.0    0.111691    3.886474
PIDILITIND       10.0    0.109539    2.940867
ZEEL             10.0    0.106567    4.159094
BAJAJ_AUTO       10.0    0.103083    2.903991
ULTRACEMCO       10.0    0.094578    4.019492
GODREJCP         10.0    0.087815    3.602002
BRITANNIA        10.0    0.084024    2.847359
RELIANCE         10.0    0.080540    2.561570
HINDALCO         10.0    0.076749 